In [ ]:
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error
import mlflow

In [ ]:
# mlflow.set_tracking_uri('sqlite:///mlflow.db')
mlflow.set_tracking_uri('http://localhost:5000')
mlflow.set_experiment('exp-2205')

In [ ]:
df = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')

df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df = df[(df.duration >= 1) & (df.duration <= 60)]

categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
df[categorical] = df[categorical].astype(str)

In [ ]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

np.sqrt(mean_squared_error(y_train, y_pred))

In [ ]:
sns.distplot(y_pred, label='prediction')
sns.distplot(y_train, label='actual')

plt.legend()

In [ ]:
sns.histplot(y_pred, label='prediction', kde=True)
sns.histplot(y_train, label='actual', kde=True)
plt.legend()
plt.show()

In [ ]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [ ]:
df_train = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2021-02.parquet')

In [ ]:
len(df_train), len(df_val)

In [ ]:
df_train['PU_DO'] = df_train['PULocationID'] + '_' + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + '_' + df_val['DOLocationID']

In [ ]:
categorical = ['PU_DO']
numerical = ['trip_distance']

dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [ ]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [ ]:
X_val.shape

In [ ]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)

np.sqrt(mean_squared_error(y_val, y_pred))

In [ ]:
with open('models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)

In [ ]:
with mlflow.start_run():

    mlflow.set_tag('developer', 'waseem')
    mlflow.log_param('train-data', 'green_tripdata_2021-01.parquet')
    mlflow.log_param('valid-data', 'green_tripdata_2021-02.parquet')

    alpha = 0.01
    mlflow.log_param('alpha', alpha)

    lr = Lasso(alpha)
    lr.fit(X_train, y_train)

    y_pred = lr.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))    
    mlflow.log_metric('RMSE', rmse)

    mlflow.log_artifact(local_path='models/lin_reg.bin', artifact_path='models_pickle')

## Module 2

In [ ]:
import xgboost as xgb
# from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
# from hyperopt.pyll import scope

In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
# def objective(params):
#     with mlflow.start_run():
#         mlflow.set_tag('model','xgboost')
#         mlflow.log_params(params)
#         booster = xgb.train(
#             params=params,
#             dtrain=train,
#             num_boost_round=1000,
#             evals=[(valid, 'validation')],
#             early_stopping_rounds=50
#         )
#         y_pred = booster.predict(valid)
#         rmse = np.sqrt(mean_squared_error(y_val, y_pred))
#         mlflow.log_metric('rmse',rmse)
#     return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
# search_space = {
#     # 'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
#     'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
#     'learning_rate': hp.loguniform('learning_rate', -3, 0),
#     'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
#     'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
#     'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
#     'objective': 'reg:linear',
#     'seed': 42
# }

In [ ]:
# best_result = fmin(
#     fn=objective,
#     space=search_space,
#     algo=tpe.suggest,
#     max_evals=50,
#     trials=Trials()
# )

In [ ]:
# import mlflow.xgboost
# import xgboost as xgb

with mlflow.start_run():

    best_params = {
        'learning_rate':0.461341010616988,
        'max_depth':16,
        'min_child_weight':0.6663060298762274,
        'objective':'reg:squarederror',
        'reg_alpha':0.14566888843218836,
        'reg_lambda':0.01467369193843644,
        'seed':1
    }

    mlflow.log_params(best_params)

    booster = xgb.train(
            params=best_params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
    
    with open('models/preprocessor.b', 'wb') as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact(local_path='models/preprocessor.b', artifact_path='preprocessor')

    
    y_pred = booster.predict(valid)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    mlflow.log_metric('rmse', rmse)

    mlflow.xgboost.log_model(booster, artifact_path='models_mlflow')

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor

In [ ]:
import mlflow.sklearn


with mlflow.start_run():
    rs = 1
    mlflow.log_param('random_state',rs)
    mlflow.set_tag('sample_size', '500')
    model_hist = HistGradientBoostingRegressor(random_state=rs)
    model_hist.fit(X_train.toarray()[:500], y_train[:500]) # for performance
    
    pred = model_hist.predict(X_val.toarray()[:500])
    rmse = np.sqrt(mean_squared_error(y_val[:500], pred))
    mlflow.log_metric('RMSE', rmse)
    
    # with open('models/hist_gradient_boosting.bin', 'wb') as f:
    #     pickle.dump(model_hist, f)
    
    # mlflow.log_artifact(local_path='models/hist_gradient_boosting.bin', artifact_path='HistGBRegressor')
    
    mlflow.sklearn.log_model(model_hist, artifact_path='HistGBRegressor')
    

In [ ]:
import mlflow
logged_model = 'runs:/2dc8c791b24e4ba4a9b6a29e7095cc12/HistGBRegressor'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Predict on a Pandas DataFrame.
import pandas as pd
loaded_model.predict(X_val.toarray()[:500])

In [ ]:
import mlflow
logged_model = 'runs:/7f5154d9e5ca43f4bf779b015d20ecdb/models_mlflow'

loaded_model = mlflow.pyfunc.load_model(logged_model)

In [ ]:
loaded_model

In [ ]:
import mlflow.xgboost


xgboost_model = mlflow.xgboost.load_model(logged_model)

In [ ]:
xgboost_model

In [ ]:
xgboost_model